### Based on Context Window(Co - Occurence)

In [18]:
import pandas as pd
from collections import defaultdict

### 텍스트 읽어오기 

In [29]:
with open('./review.sorted.uniq.refined.tsv.text.tok') as f:
    lines = [l.strip() for l in f.read().splitlines() if l.strip()]
# 데이터에서 라벨은 제외한 텍스트만 불러오기 

### 방법 정의

In [30]:
def get_term_frequency(document):
    term_freq = {}
    # 단어와 빈도수를 보여주기 위해 dict형태 
    words = document.split()
    # 단어들을 빈칸으로 split
    
    for w in words:
        term_freq[w] = 1 + (0 if term_freq.get(w) is None else term_freq[w])
        # 만약 word에 있는 단어가 존재하면 1씩 더해주고 단어가 존재하지 않으면 더해주지 않는다 
        
    return term_freq

In [31]:
def get_context_counts(lines, vocab, w_size = 2):
    context_cnt = defaultdict(int)
    
    for line in lines:
        words = line.split()
        # 단어를 split
        
        for i, w in enumerate(words):
            if w in vocab:
                # 단어가 있으면 
                for c in words[i - w_size:i + w_size]:
                    if w != c:
                        context_cnt[(w,c)] += 1
                    # w_size가 2니깐 주위 2칸씩 돌면서 count 
    return context_cnt

# context_cnt는 key가 (w,c)이고 value가 int

In [32]:
def get_co_occurences_df(context_cnt, vocab):
    data = []
    
    for word1 in vocab:
        row = []
        
        for word2 in vocab:
            try:
                count = context_cnt[(word1, word2)]
            except KeyError:
                count = 0
            row.append(count)
            
        data.append(row)
        
    return pd.DataFrame(data, index = vocab, columns = vocab)

### Call Methods

In [33]:
term_freq = pd.Series(
    get_term_frequency(' '.join(lines))
).sort_values(ascending = False)

print(term_freq)

# 데이터안의 각 단어들의 빈도를 보여줌 

.         86303
고         49631
이         44952
하         42916
좋         34589
          ...  
두드려서          1
gvudsf        1
ㅕㅅ            1
나후            1
녹물            1
Length: 30084, dtype: int64


In [34]:
vector_size = 800
term_freq.index[:vector_size]
# 총 3만개의 데이터 중 빈도가 그나마 높다고 생각되는 단어만 추출하는 것(약 800개로 추정)

Index(['.', '고', '이', '하', '좋', '네요', '도', '에', '는', '가',
       ...
       '한쪽', '엄마', '가을', 'ㅁ', '국산', '요청', '마', '보풀', '세일', '싸구려'],
      dtype='object', length=800)

In [35]:
term_freq

.         86303
고         49631
이         44952
하         42916
좋         34589
          ...  
두드려서          1
gvudsf        1
ㅕㅅ            1
나후            1
녹물            1
Length: 30084, dtype: int64

In [36]:
context_cnt = pd.Series(
    get_context_counts(
        lines,
        term_freq.index[:vector_size],
        w_size  = 4
    )
)

context_cnt

라고  비지떡     31
    ".       1
    200      2
    ml       5
    판매      16
          ... 
았   ㅍ        1
감사  ㅍㅍ       2
고   수수     106
수고  수수     212
    고수       3
Length: 1047278, dtype: int64

In [37]:
context_cnt

라고  비지떡     31
    ".       1
    200      2
    ml       5
    판매      16
          ... 
았   ㅍ        1
감사  ㅍㅍ       2
고   수수     106
수고  수수     212
    고수       3
Length: 1047278, dtype: int64

In [41]:
df = get_co_occurences_df(context_cnt, term_freq.index[:vector_size])

df
# 단어와 단어간 같이 나온 빈도를 확인할 수 있음 
# 이거 좋네
# 이걸보고 어떻게 전처리할지 판단할 수 있겠다
# 특이점은 tail부분에는 sparse한 부분이 많은 
# pca에 넣으면 괜찮은 vector가 나올 듯 

,.,고,이,하,좋,네요,도,에,는,가,...,한쪽,엄마,가을,ㅁ,국산,요청,마,보풀,세일,싸구려
.,0,9200,11111,9760,8211,14149,7929,6816,5063,5321,...,41,45,63,14,44,68,30,61,35,85
고,8762,0,4682,13988,7102,3398,15105,3634,3142,2426,...,62,15,26,17,23,30,47,41,18,30
이,10028,4767,0,3377,2854,6044,2169,3326,4554,2224,...,140,18,35,12,101,14,21,128,51,42
하,9051,13688,3625,0,3654,5694,4859,2738,5121,1925,...,19,40,9,20,13,63,30,18,60,15
좋,8096,8274,2990,3881,0,5551,5104,1636,1194,1210,...,0,28,33,23,15,0,9,11,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
요청,68,29,18,59,1,11,17,33,4,14,...,0,0,0,0,0,0,0,0,0,0
마,32,51,19,35,8,37,12,14,10,13,...,0,0,0,0,0,0,0,0,0,0
보풀,54,46,124,23,14,36,26,38,22,6,...,0,0,0,0,0,0,0,0,0,0
세일,37,25,22,50,12,13,10,22,18,5,...,0,0,0,0,22,0,0,0,0,0


In [44]:
print(df.values)
print(df.values.shape)

[[    0  9200 11111 ...    61    35    85]
 [ 8762     0  4682 ...    41    18    30]
 [10028  4767     0 ...   128    51    42]
 ...
 [   54    46   124 ...     0     0     0]
 [   37    25    22 ...     0     0     0]
 [   66    31    39 ...     0     0     0]]
(800, 800)


### 단어간 유사도 측정 

In [45]:
import torch 

In [46]:
def get_l1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()
# l1은 절댓값에 합 

In [47]:
def get_l2_distance(x1, x2):
    return ((x1 - x2) ** 2).sum()** .5
# l2는 제곱의 총합에 루트

In [48]:
def get_infinity_distance(x1, x2):
    return((x1 - x2).abs()).max()
# 절댓값을 씌우고 가장 큰 것 

In [50]:
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5 + 1e-10)

In [59]:
def get_nearest(query, dataframe, metric, top_k, ascending = True):
    vector = torch.from_numpy(dataframe.loc[query].values).float()
    # query에 대해 df에 들어있는 vector 값을 도출 
    distance = dataframe.apply(
    lambda x: metric(vector, torch.from_numpy(x.values).float()),
    axis = 1)
    # lambda함수를 사용해서 벡터와 x값에 대해 거리를 측정하는 함수를 불러와 값을 도출 
    top_distance = distance.sort_values(ascending = ascending)[:top_k]
    # 도출된 값에서 가장 거리가 유사한 단어를 사용자가 보고 싶어하는 수만큼 출력 
    print(', '.join([f'{k} ({v:.1f})' for k, v in top_distance.items()]))
    # key와 value를 도출하기 위해 
    
# 지정한 단어의 거리를 출력해주는 함수 

In [61]:
print('L1 distance:')
get_nearest('고', df, get_l1_distance, 10)
# '고'에서는 '도'가 l1에서 가장 거리가 가깞다

L1 distance:
고 (0.0), 도 (120802.0), 네요 (130039.0), 이 (139468.0), 는 (140378.0), 하 (142467.0), 에 (146747.0), 가 (161040.0), 은 (162995.0), 게 (168644.0)


In [63]:
print('L2 distance:')
get_nearest('고', df, get_l2_distance, 10)

L2 distance:
고 (0.0), 네요 (18046.8), 는 (20007.7), 이 (20206.4), 에 (20411.3), 게 (21296.6), 습니다 (22021.1), 은 (22074.9), 가 (22575.3), 는데 (22848.5)


In [65]:
print('Infinity distance:')
get_nearest('고', df, get_infinity_distance, 10)

Infinity distance:
고 (0.0), . (10751.0), 네요 (11739.0), 에 (12074.0), 안 (12854.0), 이 (12936.0), 배송 (12946.0), 지 (13247.0), 아요 (13264.0), 가격 (13315.0)


In [68]:
print('Cosine distance:')
get_nearest('고', df, get_cosine_similarity, 10, ascending = False)

Cosine distance:
고 (1.0), 구 (1.0), 구요 (0.9), 기 (0.8), 면서 (0.8), 다고 (0.8), 포장 (0.8), 내요 (0.8), 고요 (0.8), 여 (0.8)


### 정리 
1. 텍스트를 불러와 각 단어와 빈도수를 도출하는 함수 
2. 다양한 거리 측정함수를 사용해서 단어에 대한 거리를 측정
3. Cosine distance가 가장 실효성이 높아보인다.